<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Initializations" data-toc-modified-id="Initializations-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Initializations</a></span></li><li><span><a href="#General-Questions-and-Comments" data-toc-modified-id="General-Questions-and-Comments-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>General Questions and Comments</a></span></li><li><span><a href="#Exact-Solution-for-Convection-Diffusion-Equation" data-toc-modified-id="Exact-Solution-for-Convection-Diffusion-Equation-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Exact Solution for Convection-Diffusion Equation</a></span></li><li><span><a href="#Discontinuous-Galerkin-for-Convection-Diffusion-with-upwind" data-toc-modified-id="Discontinuous-Galerkin-for-Convection-Diffusion-with-upwind-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Discontinuous Galerkin for Convection-Diffusion with upwind</a></span></li><li><span><a href="#Hybrid-Discontinuous-Galerkin-Convection-Diffusion-with-upwind" data-toc-modified-id="Hybrid-Discontinuous-Galerkin-Convection-Diffusion-with-upwind-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Hybrid Discontinuous Galerkin Convection-Diffusion with upwind</a></span></li><li><span><a href="#Solving-Mass-Matrix-problem" data-toc-modified-id="Solving-Mass-Matrix-problem-6"><span class="toc-item-num">6&nbsp;&nbsp;</span>Solving Mass Matrix problem</a></span></li><li><span><a href="#Enrichment" data-toc-modified-id="Enrichment-7"><span class="toc-item-num">7&nbsp;&nbsp;</span>Enrichment</a></span><ul class="toc-item"><li><span><a href="#Enrichment-for-Mass-Matrix" data-toc-modified-id="Enrichment-for-Mass-Matrix-7.1"><span class="toc-item-num">7.1&nbsp;&nbsp;</span>Enrichment for Mass Matrix</a></span><ul class="toc-item"><li><ul class="toc-item"><li><span><a href="#Enrichment-for-Mass-Matrix-(enrich-x-axis)" data-toc-modified-id="Enrichment-for-Mass-Matrix-(enrich-x-axis)-7.1.0.1"><span class="toc-item-num">7.1.0.1&nbsp;&nbsp;</span>Enrichment for Mass Matrix (enrich x-axis)</a></span></li><li><span><a href="#Enrichment-for-Mass-Matrix-(enrich-y-axis)" data-toc-modified-id="Enrichment-for-Mass-Matrix-(enrich-y-axis)-7.1.0.2"><span class="toc-item-num">7.1.0.2&nbsp;&nbsp;</span>Enrichment for Mass Matrix (enrich y-axis)</a></span></li><li><span><a href="#Enrichment-for-Mass-Matrix-(enrich-both-xy-axis)" data-toc-modified-id="Enrichment-for-Mass-Matrix-(enrich-both-xy-axis)-7.1.0.3"><span class="toc-item-num">7.1.0.3&nbsp;&nbsp;</span>Enrichment for Mass Matrix (enrich both xy-axis)</a></span></li></ul></li></ul></li><li><span><a href="#Enrichment-with-DG" data-toc-modified-id="Enrichment-with-DG-7.2"><span class="toc-item-num">7.2&nbsp;&nbsp;</span>Enrichment with DG</a></span></li><li><span><a href="#Enrichment-with-HDG" data-toc-modified-id="Enrichment-with-HDG-7.3"><span class="toc-item-num">7.3&nbsp;&nbsp;</span>Enrichment with HDG</a></span></li></ul></li></ul></div>

In [1]:
import netgen.gui
from netgen.geom2d import unit_square
from ngsolve import *
import pandas as pd

### Initializations

In [2]:
#epsilon = 0.01
# h = uniformly refined mesh
# order = k

In [11]:
## Parameter setup
orders = [1, 2, 3]
beta = (2,0.001)
mesh_size = [1.0, 0.5, 0.25, 0.1250, 0.0625, 0.0313]
eps = 0.01

# Exact Solution
p = lambda x: x + (exp(beta[0]*x/eps)-1)/(1-exp(beta[0]/eps))
q = lambda y: y + (exp(beta[1]*y/eps)-1)/(1-exp(beta[1]/eps))

p = lambda x: x + (exp(beta[0]*(x-1)/eps)-exp(-beta[0]/eps))/(exp(-beta[0]/eps)-1)
q = lambda y: y + (exp(beta[1]*(y-1)/eps)-exp(-beta[1]/eps))/(exp(-beta[1]/eps)-1)

exact = p(x) * q(y)

#r = lambda x: beta[1] * (1 + ((exp(beta[0] * x / eps) - 1) / (1-exp(beta[0]/eps))))
#s = lambda y: beta[0] * (y + ((exp(beta[1] * y / eps) - 1)/  (1-exp(1/eps))))

coeff =  beta[1] * p(x) +  beta[0] * q(y)
#coeff = r(x) + s(y)

coeff_x = p(x)
coeff_y = q(y)

alpha = 10

## Dataframes
columns = ['Order', 'Mesh Size', 'Error']

### General Questions and Comments

In [8]:
# 1. Check the effect of stabilization paramter on enrichment

### Exact Solution for Convection-Diffusion Equation

In [9]:
orders = 1
mesh_size = 0.05
mesh = Mesh(unit_square.GenerateMesh(maxh=mesh_size))
fes = L2(mesh, order=orders, dirichlet="bottom|right|left|top")
gfu = GridFunction(fes)
Draw(exact, mesh, 'exact')

### Discontinuous Galerkin for Convection-Diffusion with upwind

In [12]:
dg = pd.DataFrame(columns=columns)
for order in orders:
    for size in mesh_size:
        mesh = Mesh(unit_square.GenerateMesh(maxh=size))
        fes = L2(mesh, order=order, dgjumps=True)
        u, v = fes.TnT()

        jump_u = u-u.Other()
        jump_v = v-v.Other()
        n = specialcf.normal(2)
        mean_dudn = 0.5 * n * (grad(u) + grad(u.Other()))
        mean_dvdn = 0.5 * n * (grad(v) + grad(v.Other()))

        h = specialcf.mesh_size

        # diffusion
        diffusion = grad(u) * grad(v) * dx \
            +alpha * order ** 2/ h * jump_u * jump_v * dx(skeleton=True) \
            +(-mean_dudn * jump_v - mean_dvdn * jump_u) * dx(skeleton=True) \
            +alpha * order ** 2/h * u * v * ds(skeleton=True) \
            + (-n * grad(u) * v -n * grad(v) * u) * ds(skeleton=True)

        # convection
        b = CoefficientFunction((beta[0],beta[1]) )
        uup = IfPos(b * n, u, u.Other())
        convection = -b * u * grad(v) * dx + b * n * uup * jump_v * dx(skeleton=True)
        
        acd = BilinearForm(fes)
        acd += eps * diffusion + convection
        acd.Assemble()
        
        # rhs
        f = LinearForm(fes)
        f += SymbolicLFI(coeff * v)
        f.Assemble()

        gfu = GridFunction(fes, name="uDG")        
        gfu.vec.data = acd.mat.Inverse(freedofs=fes.FreeDofs(),inverse="umfpack") * f.vec
        #Draw (gfu)
        
        error = sqrt (Integrate ((gfu-exact)*(gfu-exact), mesh))
        dg.loc[len(dg)] = [order, size, error] 
        
        print ('Order:', order, 'Mesh Size:', size , "L2-error:", error)
    print('......................................................................')
    
#Draw(gfu,mesh,"gfu")
#Draw(exact, mesh, 'exact')
%store dg

Order: 1 Mesh Size: 1.0 L2-error: 0.18257309111946576
Order: 1 Mesh Size: 0.5 L2-error: 0.041359981657559976
Order: 1 Mesh Size: 0.25 L2-error: 0.008750193484002718
Order: 1 Mesh Size: 0.125 L2-error: 0.002253685406823578
Order: 1 Mesh Size: 0.0625 L2-error: 0.0005383411099783809
Order: 1 Mesh Size: 0.0313 L2-error: 0.00018108211261161925
......................................................................
Order: 2 Mesh Size: 1.0 L2-error: 0.01853983925032323
Order: 2 Mesh Size: 0.5 L2-error: 0.0048984121176918706
Order: 2 Mesh Size: 0.25 L2-error: 0.00039868236432310264
Order: 2 Mesh Size: 0.125 L2-error: 0.0002666594046657812
Order: 2 Mesh Size: 0.0625 L2-error: 0.0002309231873180117
Order: 2 Mesh Size: 0.0313 L2-error: 9.834853448542017e-05
......................................................................
Order: 3 Mesh Size: 1.0 L2-error: 0.0045750580823579546
Order: 3 Mesh Size: 0.5 L2-error: 0.0005531519948967343
Order: 3 Mesh Size: 0.25 L2-error: 0.0005412666022360885
Orde

### Hybrid Discontinuous Galerkin Convection-Diffusion with upwind

In [13]:
hdg = pd.DataFrame(columns=columns)
for order in orders:
    for size in mesh_size:
        mesh = Mesh(unit_square.GenerateMesh(maxh=size))
        V = L2(mesh, order=order)
        F = FacetFESpace(mesh, order=order, dirichlet="bottom|left|right|top")

        fes = FESpace([V,F])
        u,uhat = fes.TrialFunction()
        v,vhat = fes.TestFunction()

        jump_u = u-uhat
        jump_v = v-vhat

        condense = True
        h = specialcf.mesh_size
        n = specialcf.normal(mesh.dim)

        dS = dx(element_boundary=True)

        # diffusion
        diffusion = grad(u) * grad(v) * dx + \
            alpha * order ** 2/h * jump_u * jump_v * dS + \
            (-grad(u) * n * jump_v - grad(v) * n * jump_u) * dS

        # convection            
        b = CoefficientFunction((beta[0],beta[1])) 
        uup = IfPos(b * n, u, uhat)
        convection = -b * u * grad(v) * dx + b * n * uup * jump_v * dS

        acd = BilinearForm(fes, condense=condense)
        acd += eps * diffusion + convection
        acd.Assemble()
        
        #rhs
        f = LinearForm(fes)
        f += SymbolicLFI(coeff * v)
        f.Assemble()

        gfu = GridFunction(fes)    
        if not condense:
            inv = acd.mat.Inverse(fes.FreeDofs(), "umfpack")
            gfu.vec.data = inv * f.vec
        else:
            f.vec.data += acd.harmonic_extension_trans * f.vec

            inv = acd.mat.Inverse(fes.FreeDofs(True), "umfpack")
            gfu.vec.data = inv * f.vec

            gfu.vec.data += acd.harmonic_extension * gfu.vec
            gfu.vec.data += acd.inner_solve * f.vec
            
        error = sqrt (Integrate ((gfu.components[0] - exact)*(gfu.components[0]- exact), mesh))
        hdg.loc[len(hdg)] = [order, size, error] 
        
        print ('Order:', order, 'Mesh Size:', size , "L2-error:", error)                            
    print('......................................................................')

#Draw(gfu.components[0],mesh,"gfu")
%store hdg

Order: 1 Mesh Size: 1.0 L2-error: 0.0021388692850139966
Order: 1 Mesh Size: 0.5 L2-error: 0.001544820732068654
Order: 1 Mesh Size: 0.25 L2-error: 0.00030666786891612134
Order: 1 Mesh Size: 0.125 L2-error: 0.00026642038024526265
Order: 1 Mesh Size: 0.0625 L2-error: 0.0002569751086513119
Order: 1 Mesh Size: 0.0313 L2-error: 0.00017357829671157208
......................................................................
Order: 2 Mesh Size: 1.0 L2-error: 0.0009453965788588359
Order: 2 Mesh Size: 0.5 L2-error: 0.0006825357593876917
Order: 2 Mesh Size: 0.25 L2-error: 0.0005863220096288306
Order: 2 Mesh Size: 0.125 L2-error: 0.00040409105409262245
Order: 2 Mesh Size: 0.0625 L2-error: 0.00022932110159322742
Order: 2 Mesh Size: 0.0313 L2-error: 9.062183487974479e-05
......................................................................
Order: 3 Mesh Size: 1.0 L2-error: 0.0008425809719983898
Order: 3 Mesh Size: 0.5 L2-error: 0.0008665829734104433
Order: 3 Mesh Size: 0.25 L2-error: 0.000696399534204

### Solving Mass Matrix problem

In [14]:
mm = pd.DataFrame(columns=columns)
for order in orders:
    for size in mesh_size:
        mesh = Mesh(unit_square.GenerateMesh(maxh=size))
        fes = L2(mesh, order=order, dirichlet="bottom|right|left|top")
        u, v = fes.TnT()
             
        m = BilinearForm(fes, symmetric=False)
        m += u * v * dx
        m.Assemble()
        
        f = LinearForm(fes)
        f += SymbolicLFI(exact * v)
        f.Assemble()
        
        invmstar = m.mat.Inverse(inverse="umfpack")
        
        gfu = GridFunction(fes)        
        gfu.vec.data = invmstar * f.vec
        
        error = sqrt (Integrate ((gfu-exact)*(gfu-exact), mesh))
        mm.loc[len(mm)] = [order, size, error] 
        
        print ('Order:', order, 'Mesh Size:', size , "L2-error:", error)            
    print('......................................................................')

%store mm

Order: 1 Mesh Size: 1.0 L2-error: 0.004459175075878797
Order: 1 Mesh Size: 0.5 L2-error: 0.0027608054936083455
Order: 1 Mesh Size: 0.25 L2-error: 0.0015611084025224865
Order: 1 Mesh Size: 0.125 L2-error: 0.0009505531270130661
Order: 1 Mesh Size: 0.0625 L2-error: 0.0005425383964925524
Order: 1 Mesh Size: 0.0313 L2-error: 0.00027188060916707226
......................................................................
Order: 2 Mesh Size: 1.0 L2-error: 0.000520829059102421
Order: 2 Mesh Size: 0.5 L2-error: 0.00011315394599983957
Order: 2 Mesh Size: 0.25 L2-error: 0.00012161280758322306
Order: 2 Mesh Size: 0.125 L2-error: 0.00018388342675307545
Order: 2 Mesh Size: 0.0625 L2-error: 0.00013790607359844768
Order: 2 Mesh Size: 0.0313 L2-error: 5.835771499727928e-05
......................................................................
Order: 3 Mesh Size: 1.0 L2-error: 1.4432766278166463e-06
Order: 3 Mesh Size: 0.5 L2-error: 1.605358539406916e-06
Order: 3 Mesh Size: 0.25 L2-error: 1.770064396746836

### Enrichment

#### Enrichment for Mass Matrix

###### Enrichment for Mass Matrix (enrich x-axis)

In [15]:
emm_x = pd.DataFrame(columns=columns)
for order in orders:
    for size in mesh_size:
        eps_size = size
        #eps = 1e-8
        mesh = Mesh(unit_square.GenerateMesh(maxh=size))
        V = L2(mesh, order=order, dirichlet="bottom|right|left|top")
        Q = L2(mesh, order=0)
        
        ba_x = BitArray(Q.ndof)        
        ba_x.Clear()
        for el in Q.Elements():
            mark = False
            for v in el.vertices:
                if (mesh[v].point[0] > 1-eps_size):
                    mark = True
            for dof in el.dofs:
                ba_x[dof] = mark
        
        Qx = Compress(Q, active_dofs=ba_x)
        fes = FESpace([V, Qx])
        (us, px), (vs, qx) = fes.TnT()
        
        p = coeff_x * px
        q = coeff_x * qx

        u = us + p
        v = vs + q

        m = BilinearForm(fes, symmetric=False)
        m += SymbolicBFI(u * v).SetIntegrationRule(TRIG,IntegrationRule(TRIG,10*order))
        m.Assemble()

        f = LinearForm(fes)
        f += SymbolicLFI(exact * v).SetIntegrationRule(TRIG,IntegrationRule(TRIG,4*order))
        f.Assemble()

        invmstar = m.mat.Inverse(inverse="sparsecholesky")
        
        gfu = GridFunction(fes)
        gfu.vec.data = invmstar * f.vec
        
        u2 = gfu.components[0] + gfu.components[1] * coeff_x
        
        error = sqrt (Integrate ((u2 - exact)*(u2- exact), mesh))
        emm_x.loc[len(emm_x)] = [order, size, error] 
        
        print('Order:', order, ' Mesh Size:', size , "L2 Error:", error)
    print('...........................................................')
    

%store emm_x

Order: 1  Mesh Size: 1.0 L2 Error: 0.0029526374845880088
Order: 1  Mesh Size: 0.5 L2 Error: 0.0009501780798797169
Order: 1  Mesh Size: 0.25 L2 Error: 0.00043207219306404715
Order: 1  Mesh Size: 0.125 L2 Error: 0.00016890363456931556
Order: 1  Mesh Size: 0.0625 L2 Error: 0.00016128403722161073
Order: 1  Mesh Size: 0.0313 L2 Error: 6.772025556268478e-05
...........................................................
Order: 2  Mesh Size: 1.0 L2 Error: 0.0011181913948063661
Order: 2  Mesh Size: 0.5 L2 Error: 0.0002697525546545115
Order: 2  Mesh Size: 0.25 L2 Error: 5.1198173308907434e-05
Order: 2  Mesh Size: 0.125 L2 Error: 2.7214474147138338e-05
Order: 2  Mesh Size: 0.0625 L2 Error: 1.157249435164509e-05
Order: 2  Mesh Size: 0.0313 L2 Error: 2.2101273472686767e-06
...........................................................
Order: 3  Mesh Size: 1.0 L2 Error: 0.0005307327802861761
Order: 3  Mesh Size: 0.5 L2 Error: 0.01295466842388038
Order: 3  Mesh Size: 0.25 L2 Error: 0.003714768784741006
Ord

###### Enrichment for Mass Matrix (enrich y-axis)

In [16]:
emm_y = pd.DataFrame(columns=columns)
for order in orders:
    for size in mesh_size:
        eps_size = size
        #eps = 1e-8
        mesh = Mesh(unit_square.GenerateMesh(maxh=size))
        V = L2(mesh, order=order, dirichlet="bottom|right|left|top")
        Q = L2(mesh, order=0)
        
        ba_y = BitArray(Q.ndof)        
        ba_y.Clear()
        for el in Q.Elements():
            mark = False
            for v in el.vertices:
                if (mesh[v].point[1] > 1-eps_size):
                    mark = True
            for dof in el.dofs:
                ba_y[dof] = mark
        
        Qy = Compress(Q, active_dofs=ba_y)
        fes = FESpace([V, Qy])
        (us, py), (vs, qy) = fes.TnT()
        
        p = coeff_y * py
        q = coeff_y * qy

        u = us + p
        v = vs + q

        m = BilinearForm(fes, symmetric=False)
        m += SymbolicBFI(u * v).SetIntegrationRule(TRIG,IntegrationRule(TRIG,10*order))
        m.Assemble()

        f = LinearForm(fes)
        f += SymbolicLFI(exact * v).SetIntegrationRule(TRIG,IntegrationRule(TRIG,4*order))
        f.Assemble()

        invmstar = m.mat.Inverse(inverse="sparsecholesky")
        
        gfu = GridFunction(fes)
        gfu.vec.data = invmstar * f.vec
        
        u2 = gfu.components[0] + gfu.components[1] * coeff_y
        
        error = sqrt (Integrate ((u2 - exact)*(u2- exact), mesh))
        emm_y.loc[len(emm_y)] = [order, size, error] 
        
        print('Order:', order, ' Mesh Size:', size , "L2 Error:", error)
    print('...........................................................')
    
%store emm_y

Order: 1  Mesh Size: 1.0 L2 Error: 0.0008768977915884475
Order: 1  Mesh Size: 0.5 L2 Error: 0.0002690167423375131
Order: 1  Mesh Size: 0.25 L2 Error: 0.00015731764371525384
Order: 1  Mesh Size: 0.125 L2 Error: 0.0002271815851964982
Order: 1  Mesh Size: 0.0625 L2 Error: 0.00023015034576109834
Order: 1  Mesh Size: 0.0313 L2 Error: 0.00015173522664501868
...........................................................
Order: 2  Mesh Size: 1.0 L2 Error: 0.00034364469942444235
Order: 2  Mesh Size: 0.5 L2 Error: 0.0001266462356717909
Order: 2  Mesh Size: 0.25 L2 Error: 0.00013574786225716768
Order: 2  Mesh Size: 0.125 L2 Error: 0.00011940714290110244
Order: 2  Mesh Size: 0.0625 L2 Error: 0.00010532576442310154
Order: 2  Mesh Size: 0.0313 L2 Error: 5.1576082751876446e-05
...........................................................
Order: 3  Mesh Size: 1.0 L2 Error: 2.0672900005908353e-05
Order: 3  Mesh Size: 0.5 L2 Error: 0.0001780436263982014
Order: 3  Mesh Size: 0.25 L2 Error: 0.00021438554762774

###### Enrichment for Mass Matrix (enrich both xy-axis)

In [17]:
emm_xy = pd.DataFrame(columns=columns)
for order in orders:
    for size in mesh_size:
        eps_size = size
        #eps = 1e-8
        mesh = Mesh(unit_square.GenerateMesh(maxh=size))
        V = L2(mesh, order=order, dirichlet="bottom|right|left|top")
        Q = L2(mesh, order=0)
        
        ba_x = BitArray(Q.ndof)        
        ba_x.Clear()
        for el in Q.Elements():
            mark = False
            for v in el.vertices:
                if (mesh[v].point[0] > 1-eps_size):
                    mark = True
            for dof in el.dofs:
                ba_x[dof] = mark
        
        ba_y = BitArray(Q.ndof)        
        ba_y.Clear()
        for el in Q.Elements():
            mark = False
            for v in el.vertices:
                if (mesh[v].point[1] > 1-eps_size):
                    mark = True
            for dof in el.dofs:
                ba_y[dof] = mark
        
        Qx = Compress(Q, active_dofs=ba_x)
        Qy = Compress(Q, active_dofs=ba_y)
        
        fes = FESpace([V, Qx, Qy])

        (us, px, py), (vs, qx, qy) = fes.TnT()
        
        #coeff=exact
        p = coeff_x * px + coeff_y * py
        q = coeff_x * qx + coeff_y * qy

        u = us + p
        v = vs + q

        m = BilinearForm(fes, symmetric=False)
        m += SymbolicBFI(u * v).SetIntegrationRule(TRIG,IntegrationRule(TRIG,10*order))
        m.Assemble()

        f = LinearForm(fes)
        f += SymbolicLFI(exact * v).SetIntegrationRule(TRIG,IntegrationRule(TRIG,4*order))
        f.Assemble()

        invmstar = m.mat.Inverse(inverse="sparsecholesky")
        
        gfu = GridFunction(fes)
        gfu.vec.data = invmstar * f.vec
        
        u2 = gfu.components[0] + gfu.components[1] * coeff_x + gfu.components[2] * coeff_y
        #Draw(u2-exact,mesh, 'error')
        #Draw(gfu.components[0], mesh, 'testa')
        #Draw(gfu.components[1], mesh, 'testb')
        #Draw(gfu.components[1] * coeff, mesh, 'testc')
        error = sqrt (Integrate ((u2 - exact)*(u2- exact), mesh))
        emm_xy.loc[len(emm_xy)] = [order, size, error] 
        
        print('Order:', order, ' Mesh Size:', size , "L2 Error:", error)
    print('...........................................................')
    
Draw(u2, mesh, 'test')
%store emm_xy

Order: 1  Mesh Size: 1.0 L2 Error: 0.002488240717688645
Order: 1  Mesh Size: 0.5 L2 Error: 0.0005011136374907423
Order: 1  Mesh Size: 0.25 L2 Error: 0.0004339007414246051
Order: 1  Mesh Size: 0.125 L2 Error: 0.000168678625581678
Order: 1  Mesh Size: 0.0625 L2 Error: 0.00016124159449509533
Order: 1  Mesh Size: 0.0313 L2 Error: 6.771089057252018e-05
...........................................................
Order: 2  Mesh Size: 1.0 L2 Error: 0.0010660990044291184
Order: 2  Mesh Size: 0.5 L2 Error: 0.00027498211961041007
Order: 2  Mesh Size: 0.25 L2 Error: 5.170523452810129e-05
Order: 2  Mesh Size: 0.125 L2 Error: 2.72046565428555e-05
Order: 2  Mesh Size: 0.0625 L2 Error: 1.1517973025218707e-05
Order: 2  Mesh Size: 0.0313 L2 Error: 2.1846901121211085e-06
...........................................................
Order: 3  Mesh Size: 1.0 L2 Error: 0.0005350770871341323
Order: 3  Mesh Size: 0.5 L2 Error: 0.009198128750109337
Order: 3  Mesh Size: 0.25 L2 Error: 0.00371477535503917
Order: 3

In [10]:
## Observations
# Emriching the y axis has a better convergence rate then enriching both x and y
# umfpack fails for this problem

#### Enrichment with DG

In [11]:
## Questions to Explore (TODO)
## (a) Why is the error for mesh size 0.5 larger in this enrichment (val = 20, alpha = 100, eps = 0.01)?
## (b) why is the error for mesh size  0.5 'normal' with the "sparsecholesky" than with "umfpack" (parameters as in (a))?
## (c) What role does the bonusintorder play???
## (d) Changing the bonusintorder still has a bigger error on mesh size 05
## (e) How to check automatically which bonusintorder gives the best convergence rate.

#Observations
### Best Convergence for val=15, umfpack (smallest error rate so far)
### increasing the bousintorder doesn't 'necessarily' make the convergence better (also slower)

In [18]:
edg = pd.DataFrame(columns=columns)
from ngsolve.solvers import CG
val = 15 # bonusintorder
for order in orders:
    for size in mesh_size:
        eps_size = 0.0001*size
        #eps = 1e-8
        mesh = Mesh(unit_square.GenerateMesh(maxh=size))
        V = L2(mesh, order=order, dgjumps=True)
        Q = L2(mesh, order=0)
        
        ba_x = BitArray(Q.ndof)        
        ba_x.Clear()
        
        for el in Q.Elements():
            mark = False
            for v in el.vertices:
                if (mesh[v].point[0] > 1-eps_size):
                    mark = True
            for dof in el.dofs:
                ba_x[dof] = mark
        
        ba_y = BitArray(Q.ndof)
        ba_y.Clear()
        
        
        for el in Q.Elements():
            mark = False
            for v in el.vertices:
                if (mesh[v].point[1] > 1-eps_size):
                    mark = True
            for dof in el.dofs:
                ba_y[dof] = mark
        
        Qx = Compress(Q, active_dofs=ba_x)
        Qy = Compress(Q, active_dofs=ba_y)
        
        fes = FESpace([V, Qx, Qy], dgjumps = True)

        (us, px, py), (vs, qx, qy) = fes.TnT()
        
        #coeff=exact
        ## Enrichment
        p = (coeff_x * px) + (coeff_y * py)
        q = (coeff_x * qx) + (coeff_y * qy)

        u = us + p
        v = vs + q
        
        u_Other = us.Other() + coeff_x * px.Other() + coeff_y * py.Other()
        v_Other = vs.Other() + coeff_x * qx.Other() + coeff_y * qy.Other()
        
        jump_u = u - u_Other
        jump_v = v - v_Other
        
        n = specialcf.normal(2)
        
        grad_u = grad(us) \
        + CoefficientFunction((coeff_x.Diff(x), coeff_x.Diff(y))) * px \
        + CoefficientFunction((coeff_y.Diff(x), coeff_y.Diff(y))) * py \
        + coeff_x * grad(px) + coeff_y * grad(py)
        
        grad_v = grad(vs) \
        + CoefficientFunction((coeff_x.Diff(x), coeff_x.Diff(y))) * qx \
        + CoefficientFunction((coeff_y.Diff(x), coeff_y.Diff(y))) * qy \
        + coeff_x * grad(qx) + coeff_y * grad(qy)
        
        grad_uOther = grad(us.Other()) \
        + CoefficientFunction((coeff_x.Diff(x), coeff_x.Diff(y))) * px.Other() \
        + CoefficientFunction((coeff_y.Diff(x), coeff_y.Diff(y))) * py.Other() \
        + coeff_x * grad(px.Other()) + coeff_y * grad(py.Other())
        
        grad_vOther = grad(vs.Other()) \
        + CoefficientFunction((coeff_x.Diff(x), coeff_x.Diff(y))) * qx.Other() \
        + CoefficientFunction((coeff_y.Diff(x), coeff_y.Diff(y))) * qy.Other() \
        + coeff_x * grad(qx.Other()) + coeff_y * grad(qy.Other())
                
            
        mean_dudn = 0.5 * n * (grad_u + grad_uOther)
        mean_dvdn = 0.5 * n * (grad_v + grad_vOther)
        
        h = specialcf.mesh_size
        
        # diffusion
        diffusion = grad_u * grad_v * dx(bonus_intorder = val) \
        + alpha * order**2/h * jump_u * jump_v * dx(skeleton=True, bonus_intorder = val) \
        + (-mean_dudn* jump_v - mean_dvdn * jump_u) * dx(skeleton=True, bonus_intorder = val) \
        + alpha * order**2/h * u * v * ds(skeleton=True, bonus_intorder = val) \
        + (-n * grad_u * v - n * grad_v * u) * ds(skeleton=True, bonus_intorder = val)
                
        # convection
        b = CoefficientFunction((beta[0],beta[1]) )
        uup = IfPos(b*n, u, u_Other)
        convection = -b * u * grad_v * dx(bonus_intorder = val) + b * n * uup * jump_v * dx(skeleton=True, bonus_intorder = val)
        convection = -b * u * grad_v * dx(bonus_intorder = val) + b * n * uup * v * dx(element_boundary=True, bonus_intorder = val)
       
        acd = BilinearForm(fes)
        acd += eps * diffusion + convection
        with TaskManager:
            acd.Assemble()

        # rhs
        f = LinearForm(fes)
        f += SymbolicLFI(coeff * v, bonus_intorder = val)
        with TaskManager:
            f.Assemble()

        gfu = GridFunction(fes)
        gfu.vec.data = acd.mat.Inverse(inverse="umfpack") * f.vec
        #c = acd.mat.Inverse(inverse="sparsecholesky")
        #c = Preconditioner(acd,type="local",block=True,inverse="umfpack")
        #c.Update()
        #c.Test()
        #input("")
        #gfu.vec.data = CG(acd.mat,f.vec,c.mat)
        
        u2 = gfu.components[0] + gfu.components[1] * coeff_x + gfu.components[2] * coeff_y
        
        error = sqrt (Integrate ((u2 - exact)*(u2- exact), mesh))
        edg.loc[len(edg)] = [order, size, error] 
        
        print('Order:', order, ' Mesh Size:', size , "L2 Error:", error)
    print('.......................................................................')
        
#Draw (gfu.components[2] * coeff_y, mesh, 'test')
%store edg

Order: 1  Mesh Size: 1.0 L2 Error: 0.0011656810126104745
Order: 1  Mesh Size: 0.5 L2 Error: 0.0004612467335521036
Order: 1  Mesh Size: 0.25 L2 Error: 0.00013559421913937115
Order: 1  Mesh Size: 0.125 L2 Error: 4.352905614951439e-05
Order: 1  Mesh Size: 0.0625 L2 Error: 1.7567246216422345e-05
Order: 1  Mesh Size: 0.0313 L2 Error: 6.780074243854138e-06
.......................................................................
Order: 2  Mesh Size: 1.0 L2 Error: 0.0007415986460786634
Order: 2  Mesh Size: 0.5 L2 Error: 0.0003215088532700087
Order: 2  Mesh Size: 0.25 L2 Error: 0.00013131072641415595
Order: 2  Mesh Size: 0.125 L2 Error: 4.46549278846907e-05
Order: 2  Mesh Size: 0.0625 L2 Error: 1.2453894248558606e-05
Order: 2  Mesh Size: 0.0313 L2 Error: 3.084922743679977e-06
.......................................................................
Order: 3  Mesh Size: 1.0 L2 Error: 0.0006074459822307989
Order: 3  Mesh Size: 0.5 L2 Error: 0.0003440048143583874
Order: 3  Mesh Size: 0.25 L2 Error: 0

#### Enrichment with HDG

In [19]:
ehdg = pd.DataFrame(columns=columns)
val = 15
#coeff_x = sin(x)
#coeff_y = cos(y)
for order in orders:
    for size in mesh_size:        
        mesh = Mesh(unit_square.GenerateMesh(maxh=size))
        V = L2(mesh, order=order)
        
        Q = L2(mesh, order=0)
        QF = FacetFESpace(mesh, order=0)
        
        eps_size = 0.0001*size
        
        ba_x = BitArray(Q.ndof)        
        ba_x.Clear()
        
        for el in Q.Elements():
            mark = False
            for v in el.vertices:
                if (mesh[v].point[0] > 1-eps_size):
                    mark = True
            for dof in el.dofs:
                ba_x[dof] = mark
        
        ba_y = BitArray(Q.ndof)
        ba_y.Clear()
        
        for el in Q.Elements():
            mark = False
            for v in el.vertices:
                if (mesh[v].point[1] > 1-eps_size):
                    mark = True
            for dof in el.dofs:
                ba_y[dof] = mark
        
        
        gfF = GridFunction(QF)
        
        gfF.vec[:] = 0
        for el in QF.Elements():
            if ba_x[el.nr]:
                for dof in el.dofs:
                    gfF.vec[dof] += 1
        ba_F_x = BitArray(QF.ndof)
        ba_F_x.Clear()
        for i in range(QF.ndof):
            if gfF.vec[i] > 1.5:
                ba_F_x[i] = True

        gfF.vec[:] = 0
        for el in QF.Elements():
            if ba_y[el.nr]:
                for dof in el.dofs:
                    gfF.vec[dof] += 1
        ba_F_y = BitArray(QF.ndof)
        ba_F_y.Clear()
        for i in range(QF.ndof):
            if gfF.vec[i] > 1.5:
                ba_F_y[i] = True
        
        Qx = Compress(Q, active_dofs=ba_x)
        Qy = Compress(Q, active_dofs=ba_y)
        QFx = Compress(QF, active_dofs=ba_F_x)
        QFy = Compress(QF, active_dofs=ba_F_y)
        
        print(Qx.ndof,Qy.ndof,QFx.ndof,QFy.ndof)
        # Facets. Here the jumps aren't necessary. 
        F = FacetFESpace(mesh, order=order, dirichlet=".*")
        fes = FESpace([V, Qx, Qy, F, QFx, QFy])
        #print(fes.FreeDofs())

        (us, px, py, uhat, uhatx, uhaty), (vs, qx, qy, vhat, vhatx, vhaty) = fes.TnT()
        
        #coeff=exact
        p = (coeff_x * px) + (coeff_y * py) 
        q = (coeff_x * qx) + (coeff_y * qy)

        u = us + p
        v = vs + q
        
        vhat = vhat + vhatx * coeff_x + vhaty * coeff_y
        uhat = uhat + uhatx * coeff_x + uhaty * coeff_y
        #vhat = vhat + vhatx * sin(x) + vhaty * sin(y)
        #uhat = uhat + uhatx * sin(x) + uhaty * sin(y)
        
        jump_u = u-uhat
        jump_v = v-vhat
        
        grad_u = grad(us) \
        + CoefficientFunction((coeff_x.Diff(x), coeff_x.Diff(y))) * px \
        + CoefficientFunction((coeff_y.Diff(x), coeff_y.Diff(y))) * py \
        + coeff_x * grad(px) + coeff_y * grad(py)
        
        grad_v = grad(vs) \
        + CoefficientFunction((coeff_x.Diff(x), coeff_x.Diff(y))) * qx \
        + CoefficientFunction((coeff_y.Diff(x), coeff_y.Diff(y))) * qy \
        + coeff_x * grad(qx) + coeff_y * grad(qy)
        
        #condense = True
        
        h = specialcf.mesh_size
        n = specialcf.normal(mesh.dim)
        dS = dx(element_boundary=True, bonus_intorder=val)
                
        diffusion = grad_u * grad_v *dx(bonus_intorder=val) + \
            alpha * order**2/h*jump_u*jump_v*dS + \
            (-grad_u *n*jump_v - grad_v *n*jump_u)*dS
#         diffusionVOL = SymbolicBFI(eps*grad_u * grad_v, bonus_intorder=15)
#         diffusionEB = SymbolicBFI(eps*alpha * order**2/h*jump_u*jump_v + \
#             (-eps*grad_u *n*jump_v - eps*grad_v *n*jump_u),element_boundary=True,bonus_intorder=15,simd_evaluate=False)
        
        
        b = CoefficientFunction((beta[0],beta[1]))
        uup = IfPos(b * n, u, uhat)
        convection = -b * u * grad_v *dx(bonus_intorder=val) + b * n * uup * jump_v * dS

        acd = BilinearForm(fes)
        acd += eps * diffusion + convection
        #acd += convection
        #acd += diffusionVOL
        #acd += diffusionEB
        with TaskManager():
            acd.Assemble()
        #print(acd.mat)
        f = LinearForm(fes)
        f += SymbolicLFI(coeff*v,bonus_intorder=val)
        with TaskManager():
            f.Assemble()

        gfu = GridFunction(fes)
        ## umfpack gives weird values
        gfu.vec.data = acd.mat.Inverse(freedofs=fes.FreeDofs(),inverse="umfpack") * f.vec
        u2 = gfu.components[0] + gfu.components[1] * coeff_x + gfu.components[2] * coeff_y
        
        error = sqrt (Integrate ((u2 - exact)*(u2- exact), mesh, order=15))
        ehdg.loc[len(ehdg)] = [order, size, error] 
        #Draw(u2,mesh,"u2")
        #
        
        
        #
        
        #input("")
        print('Order:', order, ' Mesh Size:', size , "L2 Error:", error)
    print('.......................................................................')

#Draw(u2, mesh, 'hdg')
%store ehdg    

2 2 1 1
Order: 1  Mesh Size: 1.0 L2 Error: 0.0012243558735063195
3 4 2 3
Order: 1  Mesh Size: 0.5 L2 Error: 0.0004178846051950982
8 9 7 8
Order: 1  Mesh Size: 0.25 L2 Error: 0.0001498106607550561
16 17 15 16
Order: 1  Mesh Size: 0.125 L2 Error: 0.0001463298196004017
32 33 31 32
Order: 1  Mesh Size: 0.0625 L2 Error: 2.0154839880428518e-05
64 65 63 64
Order: 1  Mesh Size: 0.0313 L2 Error: 6.851321956688576e-05
.......................................................................
2 2 1 1
Order: 2  Mesh Size: 1.0 L2 Error: 0.0007705105357353488
3 4 2 3
Order: 2  Mesh Size: 0.5 L2 Error: 0.00031537386132234475
8 9 7 8
Order: 2  Mesh Size: 0.25 L2 Error: 0.00012228814692162896
16 17 15 16
Order: 2  Mesh Size: 0.125 L2 Error: 4.910479927805855e-05
32 33 31 32
Order: 2  Mesh Size: 0.0625 L2 Error: 1.4394112701684341e-05
64 65 63 64
Order: 2  Mesh Size: 0.0313 L2 Error: 3.274753555798145e-06
.......................................................................
2 2 1 1
Order: 3  Mesh Size: 1

In [18]:


Order: 1  Mesh Size: 1.0 L2 Error: 0.028388495093194802
Order: 1  Mesh Size: 0.5 L2 Error: 0.02571379811991953
Order: 1  Mesh Size: 0.25 L2 Error: 0.005904422734396558
Order: 1  Mesh Size: 0.125 L2 Error: 0.005404704164843337
Order: 1  Mesh Size: 0.0625 L2 Error: 0.005296168625879318
Order: 1  Mesh Size: 0.0313 L2 Error: 0.0053309775947820845
.......................................................................
Order: 2  Mesh Size: 1.0 L2 Error: 0.010496596293949176
Order: 2  Mesh Size: 0.5 L2 Error: 0.008728741331953643
Order: 2  Mesh Size: 0.25 L2 Error: 0.4611284976890837
Order: 2  Mesh Size: 0.125 L2 Error: 0.507603161665148
Order: 2  Mesh Size: 0.0625 L2 Error: 0.005608696604245726
Order: 2  Mesh Size: 0.0313 L2 Error: 0.005795270097610484
.......................................................................
Order: 3  Mesh Size: 1.0 L2 Error: 0.007377408955121438
Order: 3  Mesh Size: 0.5 L2 Error: 0.00950825959752038
Order: 3  Mesh Size: 0.25 L2 Error: 0.010878739030786382
Order: 3  Mesh Size: 0.125 L2 Error: 0.023907253675992394
Order: 3  Mesh Size: 0.0625 L2 Error: 0.008833036208798021
Order: 3  Mesh Size: 0.0313 L2 Error: 0.009371516463999415

SyntaxError: invalid syntax (<ipython-input-18-7494bb87c894>, line 1)

Exception in Tkinter callback
Traceback (most recent call last):
  File "/usr/lib/python3.8/tkinter/__init__.py", line 1880, in __call__
    try:
KeyboardInterrupt
